## Fine tune a Retriever on the GPL data
Follow [this guide](https://haystack.deepset.ai/tutorials/09_dpr_training) / [this guide](https://haystack.deepset.ai/tutorials/18_gpl) on training your own Retreiver. The differences here are all around whether you're better off using an EmbeddingRetreiver or a DensePassage Retriever. As part of the general evaluation of the preprocessor steps / qualititative analysis of this on a few chapters of Pale (for speed) I'll make the end decision since unfortunately the GPL done with the EmbeddingRetriever isn't heavily compatible with the DPR.

This will need to be done iteratively as 
1) my personal PC likely won't be able to handle all files
2) Colab is not likely to be able to do the fine tuning in one go. Will most likely process this one book at a time, starting with the shorter ones as a proof of concept.

#### Training Status
- Otherverse 
  - Pale (~ch 23.1)
  - Pact
  - Poke - in progress (mini)
  - Pate - done in mini batches
- Parahumans 
  - Worm
  - Ward 
  - Glowworm 
- Twig - done

In [1]:
MODEL_NAME_IN = "twig_pate_poke_adapted" # The name of the model to start training from. Can be either a local model or the starting model sentence-transformers/msmarco-distilbert-base-tas-b
FILENAME_TO_TUNE = "../GPL/glowworm-gpl-output.pkl" # filepath of the file / documents to tune on. Relative path to the /finetune/ directory this code executes in
# The retriever only uses a small handful of documents as its "test" configuration so we don't need to do anything special here.
MODEL_NAME_OUT = "twig_pate_poke_glowworm_adapted"
REMOVE_DOCUMENT_STORE = True # Needed if you want to make a new retriever document store
# The document store in this case is irrelevant so it's just a small bit of sample data

In [2]:
import os
print(os.getcwd())
# os.chdir('./drive/MyDrive/pale-companion-files/finetune/') # Colab only
if REMOVE_DOCUMENT_STORE:
  try:
    print("Deleting Document Store...")
    os.remove("training_document_store_index.db")
    os.remove("training_document_store_config.db")
    os.remove("faiss_document_store.db")
  except OSError:
      print("Tried to Delete Document Store - are you sure it exists?")

print(os.getcwd())

/home/gibson/pale-companion/src/finetune
Deleting Document Store...
/home/gibson/pale-companion/src/finetune


In [3]:
import pickle
with open(FILENAME_TO_TUNE,'rb') as f:
    questions = pickle.load(f)

In [4]:
questions['gpl_labels'][5]

{'question': 'what is a heart striped puppy',
 'pos_doc': 'Cap: I do hope you’ll step out of those shadows more, c.\nCap: it’s been interesting talking to you online and seeing how talkative you are here.\nCap: I like what you contribute when you speak up\nCurious_Cephalopod:  that’s me being an introvert.\nCap: that’s fixable\nCurious_Cephalopod: I think you badly underestimate my introversion, T…\nof5: Okay, hold on.\nCap: holding on\nCurious_Cephalopod: [error]!\nCurious_Cephalopod: [error]!\nCurious_Cephalopod: [error]!\nYou have disconnected from chat\nYou have reconnected to chat\nof5: As the person who knows capricorn best, I’m going to step in and say this convo should end.  It’s going to end in hurt feelings.\nCurious_Cephalopod: my feelings are fine.\nof5: Ok\nCap: I’m fine.\nof5:  Let’s end the convo here.  We’re here to back each other up.  Let’s be more cautious than not.\nCap: sure. I’m annoyed but I can take a break.\nCap has left the conversation.\nof5: I hope I didn’t 

The expected format of the data in Haystack should be:
```
[{'question', 'pos_doc', 'neg_doc', 'score'}]
```

In [5]:
!nvidia-smi

Wed Feb 22 19:16:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8    N/A /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# Colab only
# !pip install "faiss-gpu>=1.6.3,<2"
# !pip install -q git+https://github.com/deepset-ai/haystack.git


In [7]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [8]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from haystack import Document
from haystack.nodes import PreProcessor
from haystack.nodes.retriever import EmbeddingRetriever
from haystack.document_stores import FAISSDocumentStore


In [9]:
with open('../../data/chapter_fmt_list.pkl','rb') as f:
    pale_chapters = pickle.load(f)
limited_chapters = [i for i in pale_chapters if int(i['meta']['arc_number']) < 1]

chapter_documents = [Document.from_dict(d) for d in limited_chapters]
len(chapter_documents)

2

In [10]:
word_preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=300,
    split_respect_sentence_boundary=True,
    split_overlap=40,
    progress_bar=True, 
    add_page_number=True
)

corpus =  word_preprocessor.process(chapter_documents)

Preprocessing:   0%|          | 0/2 [00:00<?, ?docs/s]

In [11]:
if REMOVE_DOCUMENT_STORE: # just a cold start
  document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", similarity="cosine") # You'll need to delete the index time each time
  document_store.write_documents(corpus)
else:
  document_store = FAISSDocumentStore.load(index_path="training_document_store_index.db", config_path="training_document_store_config.db")

max_seq_length = 400 # This should be kept at 400 to match all other iterations of fine tuning
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=MODEL_NAME_IN, # In the future we may want to fine tune the embedding model on the main documents as well
    model_format="sentence_transformers",
    max_seq_len=max_seq_length,
    progress_bar=True,
)

document_store.update_embeddings(retriever) # We want to update embeddings each time though as we have a new embedding model (courtesy of the training process) and just want to do a quick timecheck. original - 24 seconds

document_store.save(index_path="training_document_store_index.db", config_path="training_document_store_config.db")


Writing Documents:   0%|          | 0/31 [00:00<?, ?it/s]

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model twig_pate_poke_adapted
INFO - haystack.document_stores.faiss -  Updating embeddings for 31 docs...


Updating Embedding:   0%|          | 0/31 [00:00<?, ? docs/s]

INFO - haystack.schema -  Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO - haystack.schema -  Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO - haystack.schema -  Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO - haystack.schema -  Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO - haystack.schema -  Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO - haystack.schema -  Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO - hay

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
num_warmup_steps = 0 # int(0.05*len(questions["gpl_labels"]))
learning_rate = 2e-5 # The recommended weight from BERT, too high and you overwrite pretrained knowledge
batch_size = 1 # 16 is the default
mini_batch_size = 4
len(questions["gpl_labels"]), int(0.05*len(questions["gpl_labels"]))

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

if mini_batch_size != 'all':
    batches = [i for i in batch(questions["gpl_labels"], n=mini_batch_size)]
else:
    batches = [questions["gpl_labels"]]
len(batches)

252

In [13]:
for i, batch in enumerate(batches):
    print(f"{i+1} of {len(batches)}")
    retriever.train(batch, n_epochs=1,num_warmup_steps=num_warmup_steps, learning_rate=learning_rate)

INFO - haystack.nodes.retriever._embedding_encoder -  Training/adapting SentenceTransformer(
  (0): Transformer({'max_seq_length': 400, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) with 4 examples


1 of 252


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration: 0it [00:00, ?it/s]

INFO - haystack.nodes.retriever._embedding_encoder -  Training/adapting SentenceTransformer(
  (0): Transformer({'max_seq_length': 400, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) with 4 examples


...

Iteration: 0it [00:00, ?it/s]

INFO - haystack.nodes.retriever._embedding_encoder -  Training/adapting SentenceTransformer(
  (0): Transformer({'max_seq_length': 400, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) with 2 examples


252 of 252


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration: 0it [00:00, ?it/s]

In [ ]:
retriever.save(MODEL_NAME_OUT)
print(f"Underlying retriever model saved to {MODEL_NAME_OUT}")